In [11]:
########################################################
######################################################## cell type relationship inference using cell type fraction
import pandas as pd
from collections import defaultdict
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib import rc
from scipy import stats
import itertools
from scipy import linalg
from sklearn import mixture
######################################## read metadata for IPF single cell RNA-Seq data
path = 'E:\\backup\\update_data_20201210\\update_data_jupyter\\'
x = pd.read_csv(path+'GSE136831_AllCells.Samples.CellType.MetadataTable.txt',sep = '\t')

In [12]:
x.shape

(312928, 9)

In [13]:
x.iloc[0:10,]

,CellBarcode_Identity,nUMI,nGene,CellType_Category,Manuscript_Identity,Subclass_Cell_Identity,Disease_Identity,Subject_Identity,Library_Identity
0,001C_AAACCTGCATCGGGTC,5477,2150,Myeloid,ncMonocyte,Monocyte_Non-Classical,Control,001C,001C
1,001C_AAACCTGTCAACACCA,20311,4726,Myeloid,Macrophage_Alveolar,Macrophage_Alveolar,Control,001C,001C
2,001C_AAACCTGTCACAGTAC,1390,881,Lymphoid,NK,NK,Control,001C,001C
3,001C_AAACCTGTCTGTCTAT,3968,1943,Myeloid,cMonocyte,Monocyte,Control,001C,001C
4,001C_AAACGGGAGACTAAGT,3036,1716,Endothelial,Lymphatic,Lymphatic-Endothelial,Control,001C,001C
5,001C_AAACGGGAGGCTCATT,4514,1936,Myeloid,Macrophage,Macrophage,Control,001C,001C
6,001C_AAACGGGAGGGAACGG,3773,1355,Myeloid,cMonocyte,Monocyte,Control,001C,001C
7,001C_AAACGGGCAGTAGAGC,2135,991,Myeloid,cMonocyte,Monocyte,Control,001C,001C
8,001C_AAACGGGGTATAATGG,7569,2890,Myeloid,Macrophage_Alveolar,Macrophage_Alveolar,Control,001C,001C
9,001C_AAACGGGGTTTAGGAA,5755,2760,Myeloid,Macrophage,Macrophage_CellCycle_B,Control,001C,001C


In [ ]:
################### rank cell types based on their counts
x_cell_type_count = x['Manuscript_Identity'].value_counts()
cell_types_whole = x_cell_type_count.index.values

In [ ]:
cell_types_whole

In [ ]:
################ for each individual, calculate its cell type fraction
disease_state = ['Control','IPF']
ctx_list = []
for disease in disease_state:
    x1 = x[x['Disease_Identity']==disease]
    ctx = pd.crosstab(x1['Subject_Identity'], x1['Manuscript_Identity'], normalize='index')
    ctx = ctx.fillna(0)
    ctx_list.append(ctx)

ctxx = pd.concat(ctx_list,axis = 0)
# cty = pd.concat(ctx_list_list) ## 35 31 20
cty = ctxx.fillna(0)
cell_pro = pd.DataFrame(cty)
cell_pro = cell_pro.fillna(0)
disease_lis = pd.Series(np.array(['Control' for i in range(28)] + ['IPF' for i in range(32)]))
cell_pro = cell_pro.assign(disease=disease_lis.values)


In [ ]:
cell_pro.shape
cell_pro.iloc[0:10,0:10]

In [ ]:
############################################### for each cell type, using its fraction value to train one Guassian Mixture Model with two components for all individuals

X = np.log10(cell_pro.iloc[:,0:-1]+10**-2)
Y_comp = []
n_components_range = range(2, 3)
for cell_type in cell_types_whole:
    print (cell_type)
    lowest_bic = np.infty
    bic = []
    for n_components in n_components_range:
        # Fit a Gaussian mixture with EM
        gmm = mixture.GaussianMixture(n_components=n_components,max_iter=500,tol =1*10**-4)#,covariance_type='diag',tol =10**-5)#,means_init =np.array([[-1.5],[-0.5]]))
        gmm.fit(np.array(X[cell_type])[:,np.newaxis])
        bic.append(gmm.bic(np.array(X[cell_type])[:,np.newaxis]))
        if bic[-1] < lowest_bic:
            lowest_bic = bic[-1]
            best_gmm = gmm
    Y=best_gmm.predict(np.array(X[cell_type])[:,np.newaxis])
    Y_comp.append(Y)

f, axes = plt.subplots(6, 7,  sharex=True,figsize=(15, 15))
for i in range(cell_pro.shape[1]-1):
    sns.distplot( X[cell_types_whole[i]] , ax=axes[i//7, i%7])
    for j in range(Y_comp[i].max()+1):
        label_index = np.where(Y_comp[i]==j)[0]
        ax= sns.distplot(X[cell_types_whole[i]].iloc[label_index], ax=axes[i // 7, i % 7],hist = False, rug=True, label =str(j))
    # plt.setp(ax.get_legend().get_texts(), fontsize='2')  # for legend text
    # plt.setp(ax.get_legend().get_title(), fontsize='2')  # for legend title

plt.savefig(path+'\cell_type_prop_distri_GMM_log_2components_jupyter.pdf',bbox_inches='tight')
del f, axes

In [ ]:

def get_GMM_cell_pro (cell_pro,cell_types,indexx,data_path):
    X = np.log10(cell_pro.iloc[:,1:-1]+10**-2)
    Y_comp = []
    n_components_range = range(2, 3)
    for cell_type in cell_types:
        #print (cell_type)
        lowest_bic = np.infty
        bic = []
        for n_components in n_components_range:
            # Fit a Gaussian mixture with EM
            gmm = mixture.GaussianMixture(n_components=n_components,max_iter=500,tol =1*10**-4)#,covariance_type='diag',tol =10**-5)#,means_init =np.array([[-1.5],[-0.5]]))
            gmm.fit(np.array(X[cell_type])[:,np.newaxis])
            bic.append(gmm.bic(np.array(X[cell_type])[:,np.newaxis]))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmm = gmm
        Y=best_gmm.predict(np.array(X[cell_type])[:,np.newaxis])
        Y_comp.append(Y)
    np.save(data_path+'\GMM_comp_'+str(indexx)+'.npy',np.array(Y_comp))
    h_index = {}
    h_index[0]='"0"'
    h_index[1]='"1"'
    h_index[2]='"2"'
    h_index[3]='"3"'
    h_index[4]='"4"'
    Z_label= np.zeros((X.shape[0],cell_types.shape[0]))
    Z_label_df = pd.DataFrame(Z_label,columns  = cell_types )
    for i in range(cell_types.shape[0]):
        ratios =  X[cell_types[i]]
        labels = Y_comp[i]
        mean_ratios = []
        for j in range (labels.max()+1):
            mean_ratios.append(ratios.iloc[np.where(labels==j)[0]].mean())
        sort_mean_ratios = np.argsort(mean_ratios)
        for j in range(len(sort_mean_ratios)):
            Z_label_df.iloc[np.where(labels==sort_mean_ratios[j])[0],i] = h_index[j]
    Z_label_df = Z_label_df.assign(disease=cell_pro.iloc[:,-1].values)
    Z_label_df.to_csv(data_path+'\cell_pro_sort_discrete_'+str(indexx)+'.csv')
    return

In [ ]:
path

In [ ]:
########################################################  bootsrap cells for 100 times
import random
import os
folder = os.path.exists(path+'\IPF_control_cell_sampling_no_COPD')
if not folder:
    os.makedirs(path+'\IPF_control_cell_sampling_no_COPD')
whole_cell_index = range(x.shape[0])
num_selection = int(x.shape[0]*0.8)
for i in range (100):
    print (i)
    random.seed(i)
    ctx_list = []
    random_selection= random.sample(whole_cell_index, num_selection)
    y = x.iloc[random_selection,]
    for disease in disease_state:
        x1 = x[x['Disease_Identity'] == disease]
        ctx = pd.crosstab(x1['Subject_Identity'], x1['Manuscript_Identity'], normalize='index')
        ctx = ctx.fillna(0)
        ctx_list.append(ctx)
    ctxx = pd.concat(ctx_list,axis = 0)
    # cty = pd.concat(ctx_list_list) ## 39 31 20
    cty = ctxx.fillna(0)
    cell_pro = pd.DataFrame(cty)
    cell_pro = cell_pro.fillna(0)
    disease_lis = pd.Series(
        np.array(['Control' for i in range(28)] + ['IPF' for i in range(32)] ))
    cell_pro = cell_pro.assign(disease=disease_lis.values)
    cell_pro.to_csv (path+'\IPF_control_cell_sampling_no_COPD\cell_pro_'+str(i)+'.csv')


In [ ]:
data_path = path+'\IPF_control_cell_sampling_no_COPD_top_cell_types\\' ########## get 100 bootsrap GMM models
folder = os.path.exists(data_path)
if not folder:
    os.makedirs(data_path)
cell_types = cell_types_whole
for i in range (100):
    print (i)
    cell_pro_rand = pd.read_csv (path+'\IPF_control_cell_sampling_no_COPD\cell_pro_'+str(i)+'.csv')
    get_GMM_cell_pro(cell_pro_rand, cell_types, i, data_path)

In [ ]:
############### Using R to do Bayesian Network inferenece please run it using R or R studio R.3.6.1
######################### update Yale data//control / top cell types
#path = 'E:/backup/update_data_20201210/update_data_jupyter/IPF_control_cell_sampling_no_COPD_top_cell_types/'
#dir.create(paste(path,'IPF_control_cell_sampling_no_COPD_top_cell_types/Control_edges_37',sep=''))
#dir.create("E:/backup/update_data_20201210/update_data_jupyter/IPF_control_cell_sampling_no_COPD_top_cell_types/IPF_control_cell_sampling_no_COPD_top_cell_types/Control_edges_37")
#dir.create("E:/backup/update_data_20201210/update_data_jupyter/IPF_control_cell_sampling_no_COPD_top_cell_types/IPF_control_cell_sampling_no_COPD_top_cell_types/IPF_edges_37")

## please creat the dir by manual
library(bnlearn)
library(Rgraphviz)
for (j in c(0:99))
{
  print (j)
  data_path <- paste('E:/backup/update_data_20201210/update_data_jupyter/IPF_control_cell_sampling_no_COPD_top_cell_types/','cell_pro_sort_discrete_',j,'.csv',sep='')
  data <-read.csv(file = data_path,header = T)
  datax = data[1:28,2:38]
  #d = discretize(data, method = 'hartemink', breaks = 3, ibreaks = 20)
  res = rsmax2(datax)
  write.table(res$arcs, file = paste('E:/backup/update_data_20201210/update_data_jupyter/IPF_control_cell_sampling_no_COPD_top_cell_types/Control_edges_37/cell_pro_discrete_result_',j,'.txt',sep=''), sep = " ")
}

######################### update Yale data//IPF
for (j in c(0:99))
{
  print (j)

  data_path <- paste('E:/backup/update_data_20201210/update_data_jupyter/IPF_control_cell_sampling_no_COPD_top_cell_types/cell_pro_sort_discrete_',j,'.csv',sep='')
  data <-read.csv(file = data_path,header = T)
  datax = data[29:60,2:38]
  #d = discretize(data, method = 'hartemink', breaks = 3, ibreaks = 20)
  res = rsmax2(datax)
  write.table(res$arcs, file = paste('E:/backup/update_data_20201210/update_data_jupyter/IPF_control_cell_sampling_no_COPD_top_cell_types/IPF_edges_37/cell_pro_discrete_result_',j,'.txt',sep=''), sep = " ")
}

In [ ]:
###################################Back to Python
##################################  BN results analysis

import os
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from networkx.drawing.nx_agraph import graphviz_layout
import pandas as pd
from collections import defaultdict
path = "E:\\backup\\update_data_20201210\\update_data_jupyter\\IPF_control_cell_sampling_no_COPD_top_cell_types\\IPF_edges_37" #
files= os.listdir(path)
h_cell_type_pair = {}
for file in files: #
     if not os.path.isdir(file): #
        print (file)
        f = open(path+"/"+file) #
        next(f)
        for line in f: #
            info = line.strip().split()
            # print (info)
            cell_type_pair = info[1][1:-1]+'\t'+info[2][1:-1]
            if cell_type_pair not in h_cell_type_pair:
                h_cell_type_pair[cell_type_pair] = 1
            else:
                h_cell_type_pair[cell_type_pair] = h_cell_type_pair[cell_type_pair]+1
        f.close()

h_cell_type_pair_IPF = h_cell_type_pair

path = "E:\\backup\\update_data_20201210\\update_data_jupyter\\IPF_control_cell_sampling_no_COPD_top_cell_types\\Control_edges_37" #
files= os.listdir(path)
h_cell_type_pair = {}
for file in files: #
     if not os.path.isdir(file): #
        print (file)
        f = open(path+"/"+file) #
        next(f)
        for line in f: #
            info = line.strip().split()
            # print (info)
            cell_type_pair = info[1][1:-1]+'\t'+info[2][1:-1]
            if cell_type_pair not in h_cell_type_pair:
                h_cell_type_pair[cell_type_pair] = 1
            else:
                h_cell_type_pair[cell_type_pair] = h_cell_type_pair[cell_type_pair]+1
        f.close()
        
h_cell_type_pair_control = h_cell_type_pair

In [ ]:
cell_type_pair_ovlp = set(h_cell_type_pair_control.keys())| set(h_cell_type_pair_IPF.keys())
dfx = defaultdict(list)
for cell_type_pair in cell_type_pair_ovlp:
    dfx['cell_type'].append(cell_type_pair)
    if cell_type_pair in h_cell_type_pair_control:
        dfx['control'].append(h_cell_type_pair_control[cell_type_pair])
    else:
        dfx['control'].append(0)
    if cell_type_pair in h_cell_type_pair_IPF:
        dfx['IPF'].append(h_cell_type_pair_IPF[cell_type_pair])
    else:
        dfx['IPF'].append(0)
dfxx = pd.DataFrame(dfx)

dfxx.to_csv('E:\\backup\\update_data_20201210\\update_data_jupyter\\IPF_control_cell_sampling_no_COPD_top_cell_types\\top_cell_types_37_IPF_vs_Control_cell_sampling_BDN_cell_type_pair_num.csv')


In [16]:

################ we then convert the csv file to a standard file that has the following columns
"cell_type1" "cell_type2" "control" "IPF" "IPF-control" "abs(IPF-control)"
dff=pd.read_csv('E:\\backup\\update_data_20201210\\update_data_jupyter\\IPF_control_cell_sampling_no_COPD_top_cell_types\\top_cell_types_37_IPF_vs_Control_cell_sampling_BDN_cell_type_pair_num.csv')

In [18]:
dff.iloc[0:30,]

,cell_type1,cell_type2,control,IPF,diff,abs_diff
0,Macrophage,cDC1,0,100,100,100
1,VE_Capillary_B,SMC,100,0,-100,100
2,Mesothelial,Aberrant_Basaloid,0,100,100,100
3,cDC2,DC_Mature,0,100,100,100
4,ncMonocyte,Multiplet,0,100,100,100
5,VE_Peribronchial,Pericyte,0,100,100,100
6,cDC2,cDC1,100,0,-100,100
7,Ciliated,ncMonocyte,100,0,-100,100
8,B_Plasma,Mesothelial,100,0,-100,100
9,Fibroblast,Lymphatic,100,0,-100,100


In [ ]:
#########################################################################################
#########################################################################################
################ next we do ligand receptor regression analysis for the cell type pairs generated by the Bayesian Network
import pandas as pd
from collections import defaultdict
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import random
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV, LassoLarsCV, LassoLarsIC
from sklearn import datasets
from scipy import stats
import pickle
import time
import pandas as pd
#########################################
##
# we use the ligand-target interaction matrix from NicheNet, and then calculated each cell type's ligand and target average expression
# across all individuals in control case and in IPF case respectively. And  we first normalize the raw data so that the sum of expression of all genes in one cell is 1.
### the absoulte value does not matter because we are interested in expression fold change between case and control.


#ligand_target_matrix = pd.read_csv('E:\\backup\\update_data_20201210\\update_data_jupyter\\ligand_target_matrix.txt',sep=' ')
# df_expx = pd.read_csv('E:\\backup\\update_data_20201210\\update_data_jupyter\\average_gene_exp_cell_type_IPF_Control.csv')
#
# ############ gene name alignment
# ligand_list = list(ligand_target_matrix)
# target_list = list(ligand_target_matrix.index)
# ligand_list_lower = [i.lower() for i in ligand_list]
# target_list_lower = [i.lower() for i in target_list]
#
# expression_data_gene_list = df_expx['gene'][0:45947] ## target_list contains ligand list is ok, since target is in another cell
# h_expression_gene_index = {}
# for i in range (len(expression_data_gene_list)):
#     h_expression_gene_index[expression_data_gene_list[i]] = i
#
# target_in_target_index = []
# target_in_expression_index = []
# ligand_in_target_index = []
# ligand_in_expression_index = []
#
# for i in range(len(target_list_lower)):
#     if target_list_lower[i] in h_expression_gene_index:
#         target_in_target_index.append(i)
#         target_in_expression_index.append(h_expression_gene_index[target_list_lower[i]]) ### get target gene index
#
# for i in range(len(ligand_list_lower)):
#     if ligand_list_lower[i] in h_expression_gene_index:
#         ligand_in_target_index.append(i)
#         ligand_in_expression_index.append(h_expression_gene_index[ligand_list_lower[i]])  ## get ligand gene index
#
# aligned_ligand_target_matrix = ligand_target_matrix.iloc[target_in_target_index,ligand_in_target_index]
# aligned_ligand_target_matrix.to_csv('E:\\backup\\update_data_20201210\\update_data_jupyter\\ligand_target_matrix_aligned_yale.csv')
#
# aligned_df_exp_target_index = []
# for i in range (39*2): ### 39 cell types 
#     aligned_df_exp_target_index = aligned_df_exp_target_index + list(np.array(target_in_expression_index)+45947*i)
#
# aligned_df_exp_target = df_expx.iloc[aligned_df_exp_target_index,]
# aligned_df_exp_targetx = aligned_df_exp_target.fillna(0)
# aligned_df_exp_targetx.to_csv('E:\\backup\\update_data_20201210\\update_data_jupyter\\target_expression_aligned_yale.csv')
#
# aligned_df_exp_ligand_index = []
# for i in range (39*2):
#     aligned_df_exp_ligand_index = aligned_df_exp_ligand_index + list(np.array(ligand_in_expression_index)+45947*i)
#
# aligned_df_exp_ligand = df_expx.iloc[aligned_df_exp_ligand_index,]
# aligned_df_exp_ligand.to_csv('E:\\backup\\update_data_20201210\\update_data_jupyter\\ligand_expression_aligned_yale.csv')

##########################################
aligned_df_exp_ligand= pd.read_csv('E:\\backup\\update_data_20201210\\update_data_jupyter\\ligand_expression_aligned_yale.csv',index_col =0 )
aligned_df_exp_target= pd.read_csv('E:\\backup\\update_data_20201210\\update_data_jupyter\\target_expression_aligned_yale.csv',index_col =0 )
aligned_ligand_target_matrix= pd.read_csv('E:\\backup\\update_data_20201210\\update_data_jupyter\\ligand_target_matrix_aligned_yale.csv',index_col =0 )

In [ ]:
############################
############################using cell_1 ligand expression fold change to predict cell_2 target gene expression fold change
cell_type_pair_num = pd.read_csv('E:\\backup\\update_data_20201210\\update_data_jupyter\\IPF_control_cell_sampling_no_COPD_top_cell_types\\top_cell_types_37_IPF_vs_Control_cell_sampling_BDN_cell_type_pair_num.csv')
prs_list = []
for cell_pair_index in range (cell_type_pair_num.shape[0]): #### for each cell type pair
    cell_type1,cell_type2 = cell_type_pair_num['cell_type1'][cell_pair_index],cell_type_pair_num['cell_type2'][cell_pair_index]
    print (cell_type1,cell_type2)
    target_expression_fold_control = np.array(aligned_df_exp_target[(aligned_df_exp_target['cell_type']==cell_type2)&(aligned_df_exp_target['disease']=='Control')]['average'])
    target_expression_fold_disease = np.array(aligned_df_exp_target[(aligned_df_exp_target['cell_type']==cell_type2)&(aligned_df_exp_target['disease']=='IPF')]['average'])
    target_name_list = np.array(aligned_df_exp_target[(aligned_df_exp_target['cell_type'] == cell_type2) & (aligned_df_exp_target['disease'] == 'IPF')]['gene'])
    index_c = np.where(target_expression_fold_control!=0)[0]
    index_d = np.where(target_expression_fold_disease!=0)[0]
    index_cd = [i for i in index_c if i in index_d]
    if len(index_cd)>0:
        target_fold_change = target_expression_fold_disease[index_cd]/(target_expression_fold_control[index_cd])
        target_name_filter_list = target_name_list[index_cd]
        np.save('E:\\backup\\update_data_20201210\\update_data_jupyter\\IPF_control_cell_sampling_no_COPD_top_cell_types\\type_' + str(cell_pair_index) + 'target_filter.npy',target_name_filter_list)
        # target_diff_change = target_expression_fold_disease - target_expression_fold_control
        ligand_expression_fold_control = np.array(aligned_df_exp_ligand[(aligned_df_exp_ligand['cell_type']==cell_type1)&(aligned_df_exp_ligand['disease']=='Control')]['average'])
        ligand_expression_fold_disease = np.array(aligned_df_exp_ligand[(aligned_df_exp_ligand['cell_type']==cell_type1)&(aligned_df_exp_ligand['disease']=='IPF')]['average'])
        ligand_name_list = np.array(aligned_df_exp_ligand[(aligned_df_exp_ligand['cell_type'] == cell_type1) & (aligned_df_exp_ligand['disease'] == 'IPF')]['gene'])
        index_c_ligand = np.where(ligand_expression_fold_control!=0)[0]
        index_d_ligand = np.where(ligand_expression_fold_disease!=0)[0]
        index_cd_ligand = [i for i in index_c_ligand if i in index_d_ligand]
        if len(index_cd_ligand)>0:
            ligand_fold_change = ligand_expression_fold_disease[index_cd_ligand]/ligand_expression_fold_control[index_cd_ligand]
            ligand_name_filter_list = ligand_name_list[index_cd_ligand]
            np.save('E:\\backup\\update_data_20201210\\update_data_jupyter\\IPF_control_cell_sampling_no_COPD_top_cell_types\\type_' + str(cell_pair_index) + 'ligand_filter.npy',ligand_name_filter_list)
            # ligand_fold_change_filter = np.array([min(5,i) for i in ligand_fold_change])[:,np.newaxis]
            ligand_fold_change =ligand_fold_change[:,np.newaxis]
            # X_data = np.matmul(aligned_ligand_target_matrix,ligand_fold_change_filter)
            X_data = np.array(aligned_ligand_target_matrix.iloc[index_cd,index_cd_ligand])*10**3
            Y_data = np.log10(target_fold_change)
            X_data = X_data*np.log10(ligand_fold_change[:,0])
            X_data_temp = X_data+10**-10
            ##########
            ################################# regression model
            import random
            model_list = []
            length_TF = Y_data.shape[0]
            whole_data_TF = [i for i in range (length_TF)]
            random.seed(1)
            data_index = random.sample(whole_data_TF,length_TF)
            X_datax = X_data_temp[data_index]
            Y_datax = Y_data[data_index]
            for test_indel in range (1,6):
                print (test_indel)
                test_TF = [i for i in range(int(np.ceil((test_indel - 1) * 0.2 * length_TF)), int(np.ceil(test_indel * 0.2 * length_TF)))]
                train_TF = [i for i in whole_data_TF if i not in test_TF]
                X_trainx = X_datax[train_TF]
                Y_train = Y_datax[train_TF]
                std_train = X_trainx.std(axis=0) ## training normalization using parameters from training dataset
                mean_train = X_trainx.mean(axis=0)
                X_train = (X_trainx - mean_train) / std_train  ###(X_IPF+10**-10)/(X_IPF.sum(axis=0)+10**-10)
                # index3 = [i for i in index1 if i in index2]
                X_testx = X_datax[test_TF]
                X_test = (X_testx - mean_train) / std_train ## test normalization using parameters from training dataset
                Y_test = Y_datax[test_TF]
                t1 = time.time()
                model = LassoCV(cv=3, alphas=2 ** np.arange(-10, 10, 0.5),max_iter=1000).fit(X_train, Y_train)
                t_lasso_cv = time.time() - t1
                print(t_lasso_cv)
                # model.score(X_test,Y_test)
                Y_predict = model.predict(X_test)
                prsx = stats.pearsonr(Y_predict, Y_test)
                prs_list.append(prsx[0])
                model_list.append(model)
                print (prs_list)
                print (len(prs_list))
                print (cell_type1, cell_type2)
            for test_indel in range (1,6): ### five fold cross validation
                model = model_list[test_indel-1]
                m_log_alphas = -np.log2(model.alphas_)
                fig = plt.figure()
                # ymin, ymax = 2300, 3800
                plt.plot(m_log_alphas, model.mse_path_, ':')
                plt.plot(m_log_alphas, model.mse_path_.mean(axis=-1), 'k', label='Average across the folds', linewidth=2)
                plt.axvline(-np.log2(model.alpha_), linestyle='--', color='k', label='alpha: CV estimate')
                plt.legend()
                plt.xlabel('-log2(alpha)')
                plt.ylabel('Mean square error')
                plt.title('Mean square error on each fold: coordinate descent ''(train time: %.2fs)' % t_lasso_cv)
                plt.axis('tight')
                plt.savefig('E:\\backup\\update_data_20201210\\update_data_jupyter\\IPF_control_cell_sampling_no_COPD_top_cell_types\\type_'+str(cell_pair_index)+'sy_IPF_CV_10_'+str(test_indel)+'.pdf')
                del fig
            with open('E:\\backup\\update_data_20201210\\update_data_jupyter\\IPF_control_cell_sampling_no_COPD_top_cell_types\\type_'+str(cell_pair_index)+'_sy_IPF_df3_model_list.pickle', 'wb') as f:
                pickle.dump(model_list, f)
        else:
            prs_list = prs_list + [np.nan for kk in range(5)] ## failed
    else:
        prs_list = prs_list + [np.nan for kk in range (5)] ## failed

np.save('E:\\backup\\update_data_20201210\\update_data_jupyter\\IPF_control_cell_sampling_no_COPD_top_cell_types\\type_prs_list.npy',np.array(prs_list))

In [ ]:
################## analysis between  the linear regression result AND Bayesian Network
import pandas as pd
from collections import defaultdict
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import random
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV, LassoLarsCV, LassoLarsIC
from sklearn import datasets
from scipy import stats
import pickle

######### linear regression result
prs_list = np.load('E:\\backup\\update_data_20201210\\update_data_jupyter\\IPF_control_cell_sampling_no_COPD_top_cell_types\\type_prs_list.npy')
######## Bayesian Network result
cell_type_pair_num = pd.read_csv('E:\\backup\\update_data_20201210\\update_data_jupyter\\IPF_control_cell_sampling_no_COPD_top_cell_types\\top_cell_types_37_IPF_vs_Control_cell_sampling_BDN_cell_type_pair_num.csv')

prs_reshape = prs_list.reshape(int(prs_list.shape[0]/5),5)
prs_reshape_mean = prs_reshape.mean(axis=1)


prs_reshape_mean_index = []
df_prs = defaultdict(list)
for threshold in range (21):
    prs_reshape_mean_index = []
    for i in range (prs_reshape_mean.shape[0]):
        if not np.isnan(prs_reshape_mean[i]) and cell_type_pair_num['abs_diff'][i]>threshold: ## filter the result
            prs_reshape_mean_index.append(i)
    prs,pv = stats.pearsonr(prs_reshape_mean[prs_reshape_mean_index],np.log10(cell_type_pair_num['abs_diff'][prs_reshape_mean_index]))
    print (prs)
    df_prs['threshold'].append(threshold)
    df_prs['pearson corr'].append(prs)

df_prs_df = pd.DataFrame(df_prs)
df_prs_df.to_csv('E:\\backup\\update_data_20201210\\update_data_jupyter\\IPF_control_cell_sampling_no_COPD_top_cell_types\\prs_threshold.csv')

prs_reshape_mean_index = []
for i in range (prs_reshape_mean.shape[0]):
    if not np.isnan(prs_reshape_mean[i]) and cell_type_pair_num['abs_diff'][i]!=0:
        prs_reshape_mean_index.append(i)

stats.pearsonr(np.log10(prs_reshape_mean[prs_reshape_mean_index]), np.log10(cell_type_pair_num['abs_diff'][prs_reshape_mean_index]))


df = defaultdict(list)
df['log10(#edge)'] = list(np.log10(cell_type_pair_num['abs_diff'][prs_reshape_mean_index]))
df['pearson corr for test'] = list(prs_reshape_mean[prs_reshape_mean_index])
dfx = pd.DataFrame(df)
fig = plt.figure()
g=sns.jointplot('log10(#edge)','pearson corr for test',data = dfx,  kind="reg")
g.annotate(stats.spearmanr)
plt.savefig('E:\\backup\\update_data_20201210\\update_data_jupyter\\IPF_control_cell_sampling_no_COPD_top_cell_types\\cell_type_LR_vs_BN.pdf')
del fig